In [1]:
from confluent_kafka import Consumer, KafkaError
from confluent_kafka import DeserializingConsumer
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroDeserializer
from confluent_kafka.serialization import StringDeserializer

In [2]:
import yaml

In [3]:
with open("../src/config.yml", 'r') as stream:
    try:
        print(yaml.safe_load(stream))
    except yaml.YAMLError as exc:
        print(exc)

{'lsl': {'srate': 1, 'name': 'lsl-test-1', 'type': 'Test', 'n_channels': 100, 'source_id': 'lalala123', 'producer_sleep': 5, 'pulling_timeout': 10}, 'kafka': {'bootstrap_servers': 'localhost:9092', 'topic': 'lsl-test-5-avro', 'schema_registry': 'http://localhost:8081/', 'schema_path': 'schemas/lsl.sample.FloatArray.avsc', 'producer': {'client_id': 'jupyter-kafka-producer-1', 'callback_timeout': 5}, 'consumer': {'group_id': 'jupyter-kafka-consumer-1', 'auto_offset_reset': 'earliest', 'poll_timeout': 20}}}


In [4]:
topic = 'lsl-test-5-avro'
poll_timeout = 20 #sec

In [5]:
sample_schema_str = """
{
  "name": "lsl.sample.FloatArray",
  "type": "record",
  "fields": [
    {
      "name": "timestamp",
      "type": "float"
    },
    {
      "name": "sample",
      "type": {
        "type": "array",
        "items": "float"
      }
    }
  ]
}"""

In [6]:
sr_conf = {'url': "http://localhost:8081"}
schema_registry_client = SchemaRegistryClient(sr_conf)

In [7]:
avro_deserializer = AvroDeserializer(sample_schema_str,
                                         schema_registry_client)

In [8]:
consumer_conf = {'bootstrap.servers': 'localhost:9092',
                     'key.deserializer': None,
                     'value.deserializer': avro_deserializer,
                     'group.id': 'jupyter-3',
                     'auto.offset.reset': "earliest"}

In [14]:
consumer = DeserializingConsumer(consumer_conf)
consumer.subscribe([topic])

while True:
    try:
        # SIGINT can't be handled when polling, limit timeout to 1 second.
        msg = consumer.poll(1.0)
        if msg is None:
            continue
        elif msg.error() is not None:
            print(msg.error())
            raise(nsg.error())
            
        valid_msg = msg
        user = msg.value()
        if user is not None:
            print(user['timestamp'],user['sample'][0])
            
    except KeyboardInterrupt:
        break

consumer.close()

In [10]:
valid_msg.timestamp()

(1, 1615840917)

In [11]:
f = 1615840278.970943
type(f)

float

In [12]:
{"timestamp": f, "sample": 1},

({'timestamp': 1615840278.970943, 'sample': 1},)